In [81]:
using DrWatson
@quickactivate "HEMI" 
using HEMI 
using DataFrames, Chain, CSV, StringEncodings
using Plots

In [82]:
function save_csv(file::AbstractString, df::DataFrame)
    encoding = enc"ISO-8859-1"
    @info "Saving file with $encoding" file
    open(file, encoding, "w") do io 
        CSV.write(io, df)
    end
end

save_csv (generic function with 1 method)

In [83]:
savepath = mkpath(datadir("updates", "volatility"))

"c:\\Users\\RRCP\\Documents\\HEMI\\HEMI\\data\\updates\\volatility"

In [84]:
gtdata_eval = gtdata[Date(2020,12)]

UniformCountryStructure{2, Float32, Float32} con 2 bases
|─> VarCPIBase{Float32, Float32}: 120 períodos × 218 gastos básicos Jan-2001-Dec-2010
|─> VarCPIBase{Float32, Float32}: 120 períodos × 279 gastos básicos Jan-2011-Dec-2020


In [85]:
gt00gb = CSV.read(datadir("guatemala", "Guatemala_GB_2000.csv"), 
    DataFrame, types=[String, String, Float64])
gt10gb = CSV.read(datadir("guatemala", "Guatemala_GB_2010.csv"), 
    DataFrame, types=[String, String, Float64]);

## Volatilidad de optimización

In [86]:
vol_00 = std(gtdata_eval[1].v |> capitalize |> varinteran, dims=1)
vol_10 = std(gtdata_eval[2].v |> capitalize |> varinteran, dims=1)

1×279 Matrix{Float32}:
 2.59312  3.88857  9.25578  1.37753  …  2.12469  1.21573  2.49149  1.57847

## Medidas de exclusión fija óptima MSE

In [87]:
optmse2022 = wload(datadir("results", "mse-combination", "Esc-E-Scramble-OptMAI", "optmse2022", "optmse2022.jld2"), "optmse2022")
components(optmse2022)

,measure,weights
,String,Float32
1,Percentil equiponderado 72.4,4.07182e-6
2,Percentil ponderado 70.0,1.64225e-6
3,"Media Truncada Equiponderada (58.76, 83.15)",0.727412
4,"Media Truncada Ponderada (21.0, 95.89)",4.53039e-6
5,"Inflación de exclusión dinámica (0.32, 1.68)",0.0160966
6,MAI óptima MSE 2018,0.256481
7,"Exclusión fija de gastos básicos IPC (14, 14)",0.0


In [88]:
optmse_fns = optmse2022.ensemble.functions
fixed_mask = [fn isa InflationFixedExclusionCPI for fn in optmse_fns]
fix_fn = optmse_fns[fixed_mask][1]

exc_00, exc_10 = fix_fn.v_exc

([35, 30, 190, 36, 37, 40, 31, 104, 162, 32, 33, 159, 193, 161], [29, 116, 31, 46, 39, 40, 186, 30, 35, 185, 197, 34, 48, 184])

In [89]:
mse_vol_00 = copy(gt00gb)
mse_vol_00[!, :volatility] = vec(vol_00)
mse_vol_00[!, :index] = 1:218
gbvol_00 = mse_vol_00[exc_00, [:index, :GastoBasico, :volatility, :Ponderacion]]

,index,GastoBasico
,Int64,String
1,35,Cebolla
2,30,Tomate
3,190,Otras cuotas fijas y extraordinarias en la educaión preprimaria y primaria
4,36,Papa o patata
5,37,Zanahoria
6,40,Culantro o cilantro
7,31,Güisquil
8,104,Gastos derivados del gas manufacturado y natural y gases licuados del petróleo
9,162,Transporte aéreo


In [90]:
mse_vol_10 = copy(gt10gb)
mse_vol_10[!, :volatility] = vec(vol_10)
mse_vol_10[!, :index] = 1:279
gbvol_10 = mse_vol_10[exc_10, [:index, :GastoBasico, :volatility, :Ponderacion]]

,index,GastoBasico,volatility,Ponderacion
,Int64,String,Float32,Float64
1,29,Tomate,32.7544,0.92386
2,116,Gas propano,27.3757,1.19963
3,31,Chile pimiento,27.937,0.05247
4,46,Culantro,30.3996,0.03903
5,39,Cebolla,29.4752,0.33668
6,40,Papa,20.1433,0.50577
7,186,Diesel,17.4824,0.13937
8,30,Güisquil,17.5603,0.1709
9,35,Lechuga,17.1702,0.03506


In [91]:
save_csv(joinpath(savepath, "mse_00.csv"), gbvol_00)
save_csv(joinpath(savepath, "mse_10.csv"), gbvol_10)

┌ Info: Saving file with ISO-8859-1
│   file = c:\Users\RRCP\Documents\HEMI\HEMI\data\updates\volatility\mse_00.csv
└ @ Main c:\Users\RRCP\Documents\HEMI\HEMI\notebooks\historic-combination-metrics\volatility-fixed-exclusion.ipynb:3
┌ Info: Saving file with ISO-8859-1
│   file = c:\Users\RRCP\Documents\HEMI\HEMI\data\updates\volatility\mse_10.csv
└ @ Main c:\Users\RRCP\Documents\HEMI\HEMI\notebooks\historic-combination-metrics\volatility-fixed-exclusion.ipynb:3


StringEncoder{UTF-8, ISO-8859-1}(IOStream(<file c:\Users\RRCP\Documents\HEMI\HEMI\data\updates\volatility\mse_10.csv>))

## Medidas de exclusión fija óptima ME

In [92]:
optabsme2022 = wload(datadir("results", "absme-combination", "Esc-G", "optabsme2022", "optabsme2022.jld2"), "optabsme2022")
components(optabsme2022)

,measure,weights
,String,Float32
1,"Media Truncada Equiponderada (35.29, 93.4)",0.407029
2,MAI óptima de absme 2018,0.0598154
3,"Media Truncada Ponderada (34.19, 93.0)",0.133195
4,"Inflación de exclusión dinámica (1.03, 3.42)",0.152838
5,Percentil equiponderado 71.63,0.164859
6,Percentil ponderado 69.56,0.0822618
7,"Exclusión fija de gastos básicos IPC (9, 6)",0.0


In [93]:
optabsme_fns = optabsme2022.ensemble.functions
fixed_mask = [fn isa InflationFixedExclusionCPI for fn in optabsme_fns]
fix_fn = optabsme_fns[fixed_mask][1]

exc_00, exc_10 = fix_fn.v_exc

([35, 30, 190, 36, 37, 40, 31, 104, 162], [29, 116, 31, 46, 39, 40])

In [94]:
mse_vol_00 = copy(gt00gb)
mse_vol_00[!, :volatility] = vec(vol_00)
mse_vol_00[!, :index] = 1:218
gbvol_00 = mse_vol_00[exc_00, [:index, :GastoBasico, :volatility, :Ponderacion]]

,index,GastoBasico
,Int64,String
1,35,Cebolla
2,30,Tomate
3,190,Otras cuotas fijas y extraordinarias en la educaión preprimaria y primaria
4,36,Papa o patata
5,37,Zanahoria
6,40,Culantro o cilantro
7,31,Güisquil
8,104,Gastos derivados del gas manufacturado y natural y gases licuados del petróleo
9,162,Transporte aéreo


In [95]:
mse_vol_10 = copy(gt10gb)
mse_vol_10[!, :volatility] = vec(vol_10)
mse_vol_10[!, :index] = 1:279
gbvol_10 = mse_vol_10[exc_10, [:index, :GastoBasico, :volatility, :Ponderacion]]

,index,GastoBasico,volatility,Ponderacion
,Int64,String,Float32,Float64
1,29,Tomate,32.7544,0.92386
2,116,Gas propano,27.3757,1.19963
3,31,Chile pimiento,27.937,0.05247
4,46,Culantro,30.3996,0.03903
5,39,Cebolla,29.4752,0.33668
6,40,Papa,20.1433,0.50577


In [96]:
save_csv(joinpath(savepath, "absme_00.csv"), gbvol_00)
save_csv(joinpath(savepath, "absme_10.csv"), gbvol_10)

┌ Info: Saving file with ISO-8859-1
│   file = c:\Users\RRCP\Documents\HEMI\HEMI\data\updates\volatility\absme_00.csv
└ @ Main c:\Users\RRCP\Documents\HEMI\HEMI\notebooks\historic-combination-metrics\volatility-fixed-exclusion.ipynb:3
┌ Info: Saving file with ISO-8859-1
│   file = c:\Users\RRCP\Documents\HEMI\HEMI\data\updates\volatility\absme_10.csv
└ @ Main c:\Users\RRCP\Documents\HEMI\HEMI\notebooks\historic-combination-metrics\volatility-fixed-exclusion.ipynb:3


StringEncoder{UTF-8, ISO-8859-1}(IOStream(<file c:\Users\RRCP\Documents\HEMI\HEMI\data\updates\volatility\absme_10.csv>))

## Medidas de exclusión fija óptima CORR

In [97]:
optcorr2022 = wload(datadir("results", "corr-combination", "Esc-F", "optcorr2022", "optcorr2022.jld2"), "optcorr2022")
components(optcorr2022)

,measure,weights
,String,Float32
1,Percentil ponderado 80.96,3.93042e-5
2,"Inflación de exclusión dinámica (0.47, 4.97)",0.00194286
3,"Media Truncada Ponderada (46.44, 98.55)",0.0
4,Percentil equiponderado 77.25,0.19322
5,MAI óptima de correlación 2018,0.336925
6,"Media Truncada Equiponderada (55.91, 92.18)",0.467873
7,"Exclusión fija de gastos básicos IPC (12, 52)",0.0


In [98]:
optcorr_fns = optcorr2022.ensemble.functions
fixed_mask = [fn isa InflationFixedExclusionCPI for fn in optcorr_fns]
fix_fn = optcorr_fns[fixed_mask][1]

exc_00, exc_10 = fix_fn.v_exc

([35, 30, 190, 36, 37, 40, 31, 104, 162, 32, 33, 159], [29, 116, 31, 46, 39, 40, 186, 30, 35, 185  …  23, 268, 9, 36, 236, 78, 20, 213, 273, 26])

In [99]:
mse_vol_00 = copy(gt00gb)
mse_vol_00[!, :volatility] = vec(vol_00)
mse_vol_00[!, :index] = 1:218
gbvol_00 = mse_vol_00[exc_00, [:index, :GastoBasico, :volatility, :Ponderacion]]

,index,GastoBasico
,Int64,String
1,35,Cebolla
2,30,Tomate
3,190,Otras cuotas fijas y extraordinarias en la educaión preprimaria y primaria
4,36,Papa o patata
5,37,Zanahoria
6,40,Culantro o cilantro
7,31,Güisquil
8,104,Gastos derivados del gas manufacturado y natural y gases licuados del petróleo
9,162,Transporte aéreo


In [100]:
mse_vol_10 = copy(gt10gb)
mse_vol_10[!, :volatility] = vec(vol_10)
mse_vol_10[!, :index] = 1:279
gbvol_10 = mse_vol_10[exc_10, [:index, :GastoBasico, :volatility, :Ponderacion]]

,index,GastoBasico,volatility,Ponderacion
,Int64,String,Float32,Float64
1,29,Tomate,32.7544,0.92386
2,116,Gas propano,27.3757,1.19963
3,31,Chile pimiento,27.937,0.05247
4,46,Culantro,30.3996,0.03903
5,39,Cebolla,29.4752,0.33668
6,40,Papa,20.1433,0.50577
7,186,Diesel,17.4824,0.13937
8,30,Güisquil,17.5603,0.1709
9,35,Lechuga,17.1702,0.03506


In [101]:
save_csv(joinpath(savepath, "corr_00.csv"), gbvol_00)
save_csv(joinpath(savepath, "corr_10.csv"), gbvol_10)

┌ Info: Saving file with ISO-8859-1
│   file = c:\Users\RRCP\Documents\HEMI\HEMI\data\updates\volatility\corr_00.csv
└ @ Main c:\Users\RRCP\Documents\HEMI\HEMI\notebooks\historic-combination-metrics\volatility-fixed-exclusion.ipynb:3
┌ Info: Saving file with ISO-8859-1
│   file = c:\Users\RRCP\Documents\HEMI\HEMI\data\updates\volatility\corr_10.csv
└ @ Main c:\Users\RRCP\Documents\HEMI\HEMI\notebooks\historic-combination-metrics\volatility-fixed-exclusion.ipynb:3


StringEncoder{UTF-8, ISO-8859-1}(IOStream(<file c:\Users\RRCP\Documents\HEMI\HEMI\data\updates\volatility\corr_10.csv>))